# Create Xcalibur sequence

In [1]:
# INPUT
path = 'sequence_builder_sheet.xlsx'

import pandas as pd

# import excel spreadsheet, segment into bits
data = pd.read_excel(path, skiprows=1)
durs = data.iloc[0:3,1:4]
locs = data.iloc[5:,1:5].reset_index(drop=True).dropna()
locs.columns=data.iloc[4,1:5]
seq = data.iloc[:,6:8].dropna()
other_data = data.iloc[:,10]
other_data.index = data.iloc[:,9]

# import parameters
total_injection_vol = other_data.loc['injection_volume']+other_data.loc['dead_volume']
vial_bottom_vol = other_data.loc['vial_bottom_vol']
injection_dur_min = other_data.loc['injection_duration']

# identify internal standard
locs_IS = locs[locs['Type']=='Internal standard'].reset_index(drop=True)

if len(locs_IS['Name'].unique())==1:
    IS_name = locs_IS.loc[:,'Name'].iloc[0]
    IS_injections = pd.Series(name='Location')
    m=0
    for i in locs_IS.index:
        n_inj = int((locs_IS.loc[i,'Volume']-vial_bottom_vol)/total_injection_vol)
        for _ in range(n_inj):
            IS_injections[m] = locs_IS.loc[i,'Location']
            m=m+1
    print('Found internal standard',IS_name, 'at positions\n',locs_IS['Location'],'\nTotal volume',locs_IS['Volume'].sum(),'ul (',len(IS_injections),'injections)\n')
else:
    print("Couldn't identify internal standard\n")

# Build sequence
sequence = pd.DataFrame(columns = ['Sample Type','File Name','Sample ID','Path','Instrument Method','Position','Inj Vol','Sample Name'])
print('Building sequence')
# tracers
m_IS = 0
n = 0

warmupdur = durs.loc[durs['Block type'].str.startswith('Warmup'),'n_injections'].iloc[0]
warmupmeth = durs.loc[durs['Block type'].str.startswith('Warmup'),'inst_method'].iloc[0]
n_blocks = durs.loc[durs['Block type'].str.startswith('Sample'),'n_injections'].iloc[0]
blockmeth = durs.loc[durs['Block type'].str.startswith('Sample'),'inst_method'].iloc[0]
n_blanks = durs.loc[durs['Block type'].str.startswith('Blank'),'n_injections'].iloc[0]
blankmeth = durs.loc[durs['Block type'].str.startswith('Blank'),'inst_method'].iloc[0]

# add warmup blocks
for _ in range(warmupdur):
    # add IS
    sequence.loc[n,['Sample Type','Sample Name','Position','Instrument Method']] = ['Internal standard',IS_name,IS_injections[m_IS],warmupmeth]
    m_IS+=1
    n+=1
    locs.loc[locs['Location']==IS_injections[m_IS],['Volume']]-=total_injection_vol
    if locs.loc[locs['Location']==IS_injections[m_IS]].loc[:,'Volume'].iloc[0]<vial_bottom_vol:
        print('Warning: vial empty for',IS_injections[m_IS])
        
# build injection list
samples_list=[]
for i in seq.index:
    name=[seq.loc[i,'Name']]
    num=int(seq.loc[i,'Num'])
    samples_list+=num*name
# randomize order of list
import random
random.shuffle(samples_list)

# work through randomized list
for name in samples_list:
    # find vial location
    if len(locs[locs['Name']==name])==1:
        row = locs.loc[locs['Name']==name]
        
        if sequence['Sample Name'].iloc[-1] != IS_name:
            # add IS
            sequence.loc[n,['Sample Type','Sample Name','Position','Instrument Method']] = ['Internal standard', IS_name , IS_injections[m_IS], blockmeth ]
            n+=1
            m_IS+=1
            locs.loc[locs['Location']==IS_injections[m_IS],['Volume']]-=total_injection_vol
            if locs.loc[locs['Location']==IS_injections[m_IS]].loc[:,'Volume'].iloc[0]<vial_bottom_vol:
                print('Warning: vial empty for',IS_injections[m_IS])
           
        # add sample
        sequence.loc[n,['Sample Type','Sample Name','Position','Instrument Method']] = [row['Type'].iloc[0], row['Name'].iloc[0] , row['Location'].iloc[0], blockmeth ]
        n+=1
        locs.loc[locs['Name']==name,['Volume']]-=total_injection_vol
        if locs.loc[locs['Name']==name].loc[:,'Volume'].iloc[0]<vial_bottom_vol:
            print('Warning: vial empty for',IS_injections[m_IS])
        # add IS
        sequence.loc[n,['Sample Type','Sample Name','Position','Instrument Method']] = ['Internal standard', IS_name , IS_injections[m_IS], blockmeth ]
        n+=1
        m_IS+=1
        locs.loc[locs['Location']==IS_injections[m_IS],['Volume']]-=total_injection_vol
        if locs.loc[locs['Location']==IS_injections[m_IS]].loc[:,'Volume'].iloc[0]<vial_bottom_vol:
            print('Warning: vial empty for',IS_injections[m_IS])
                    
# add other colummns
path_col = other_data.loc['path']
filename = [other_data['name_prefix']+f"{id+1:003}" for id in sequence.index]
s_id = sequence.index+1
inj_vol = other_data.loc['injection_volume']

sequence.loc[:,'Path'] = path_col
sequence.loc[:,'File Name'] = filename
sequence.loc[:,'Sample ID'] = s_id
sequence.loc[:,'Inj Vol'] = inj_vol


sequence_dur_hrs = (n-1)*injection_dur_min/60

print('\nSequence duration:',sequence_dur_hrs,'hours')

print('\nRemaining volumes:\n',locs)

# save sequence as csv
sequence.to_csv('sequence.csv',index=False)

# add Bracket Type = 4 to start of file
file_path = 'sequence.csv'
with open(file_path, 'r') as file:
    content = file.readlines()
content.insert(0, 'Bracket Type=4,,,,,,,\n')
with open(file_path, 'w') as file:
    file.writelines(content)

Found internal standard VS-IS at positions
 0    B:F9
1    B:F2
2    B:F3
3    B:F4
4    B:F5
5    B:F6
6    B:F7
7    B:F8
8    B:F9
Name: Location, dtype: object 
Total volume 9000 ul ( 288 injections)

Building sequence

Sequence duration: 46.2 hours

Remaining volumes:
 4                Type      Name Location Volume
0   Internal standard     VS-IS     B:F9     70
1   Internal standard     VS-IS     B:F2     40
2   Internal standard     VS-IS     B:F3     40
3   Internal standard     VS-IS     B:F4    250
4   Internal standard     VS-IS     B:F5   1000
5   Internal standard     VS-IS     B:F6   1000
6   Internal standard     VS-IS     B:F7   1000
7   Internal standard     VS-IS     B:F8   1000
8   Internal standard     VS-IS     B:F9     70
9               Blank      MeOH     B:C1    470
10              Blank      MQ-0     B:C2    470
11              Blank      MQ-1     B:C3    470
12              Blank      MQ-2     B:C4    470
13           Standard      S1-1     B:D1    320
14   

In [7]:
sequence.head(50)

,Sample Type,File Name,Sample ID,Path,Instrument Method,Position,Inj Vol,Sample Name
0,Internal standard,250123_001,1,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
1,Internal standard,250123_002,2,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
2,Internal standard,250123_003,3,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
3,Internal standard,250123_004,4,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
4,Internal standard,250123_005,5,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
5,Internal standard,250123_006,6,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
6,Internal standard,250123_007,7,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
7,Internal standard,250123_008,8,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
8,Standard,250123_009,9,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:D2,25,S2-1
9,Internal standard,250123_010,10,C:\Xcalibur\data\DC aerosol\250123_DC-2011-2_M...,C:\Xcalibur\methods\Vanquish_Neo\IFI_A50-50B\S...,R:F1,25,VS-IS
